In [10]:
import scala.io.Source
import scala.math.BigDecimal


val serialSrc = Source.fromFile("predictions_1.tsv").getLines
val parallelSrc = Source.fromFile("predictions_40.csv").getLines
val serial = serialSrc.map { line =>
    val spl = line.split("\t")
    val id = spl(4)
    val pv = spl(5).split(", ").map(_.replace("{1=","")).map(_.replace("0=","")).map(_.replace("}",""))
    (id,pv(1),pv(0))
}
val parallel = parallelSrc.map { line =>
    val spl = line.split(",")
    (spl(0).replace("\"",""),spl(1),spl(2))
}
(parallel.toArray.sorted,serial.toArray.sorted).zipped.map {
    case((ida,pv0a,pv1a),(idb,pv0b,pv1b)) => 
        ida == idb && 
        BigDecimal(pv0a.toDouble - pv0b.toDouble).setScale(3, BigDecimal.RoundingMode.HALF_UP).toDouble.abs <= 0.001 &&
        BigDecimal(pv1a.toDouble - pv1b.toDouble).setScale(3, BigDecimal.RoundingMode.HALF_UP).toDouble.abs <= 0.001
}.reduce(_&&_)

true